# ISNARTECH POSE ESTIMATION ASSIGNMENT, TRISHIT

## importing and installing dependencies

In [70]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [71]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## keypoints using mp holistics

In [72]:
mp_holistic = mp.solutions.holistic # Holistic model to make prediction

mp_drawing = mp.solutions.drawing_utils # Drawing utilities to draw them

In [73]:
def mediapipe_detection(image, model):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    # color conversion 
    
    image.flags.writeable = False                     # image not writable to save memory
    
    results = model.process(image)
    
    image.flags.writeable = True                      # make image writeable
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # color conversion 
    
    return image, results

In [74]:
def draw_landmarks(image, results):
    
    # Draw face connections
    
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    
    
    # Draw pose connections
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    
    
    # Draw left hand connections
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    
    
    # Draw right hand connections  
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [76]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


## extracting keypoint values

In [77]:
def extract_keypoints(results):
    
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

## setup folders for collection

In [78]:
DATA_PATH = os.path.join('MP_DATA_2')

actions = np.array(['sitting actively', 'feeling sleepy', 'smoking cigarette', 'eating food'])

no_sequences = 30   # data worth 30 videos

sequence_length = 30   # each video has 30 frames

In [79]:
for action in actions:
    
    for sequence in range(no_sequences):
    
        try:
    
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
    
        except:
    
            pass

## collecting keypoint values for training and testing

In [80]:
cap = cv2.VideoCapture(0)
  
 # setting mediapipe model   
    
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # loop through the action
    for action in actions:
       
        # loop through videos
       
        for sequence in range(no_sequences):
            # loop through the frames
       

            for frame_num in range(sequence_length):

             
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                

                # Draw landmarks
                draw_landmarks(image, results)
                
                # waiting logic
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
       
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                   
                    
                    cv2.imshow('OpenCV Feed', image)
       
                    cv2.waitKey(2000)
                
                else: 
       
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Show to screen
       
                    cv2.imshow('OpenCV Feed', image)
                
                
                keypoints = extract_keypoints(results)
       
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
       
                np.save(npy_path, keypoints)

               
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## preprocess data and create feature labels

In [81]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [82]:
label_map = {label:num for num, label in enumerate(actions)}

In [83]:
sequences, labels = [], []

for action in actions:

    for sequence in range(no_sequences):

        window = []

        for frame_num in range(sequence_length):

            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))

            window.append(res)

        sequences.append(window)

        labels.append(label_map[action])

In [84]:
X = np.array(sequences)

In [85]:
y = to_categorical(labels).astype(int)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Build and Train LSTM Neural Network

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [88]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [89]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))

model.add(LSTM(128, return_sequences=True, activation='relu'))

model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

In [90]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [91]:
model.fit(X_train, y_train, epochs = 170, callbacks = [tb_callback])

Epoch 1/170
4/4 [==============================] - 4s 281ms/step - loss: 4.0431 - categorical_accuracy: 0.2614
Epoch 2/170
4/4 [==============================] - 0s 71ms/step - loss: 1.9030 - categorical_accuracy: 0.2922
Epoch 3/170
4/4 [==============================] - 0s 69ms/step - loss: 5.0329 - categorical_accuracy: 0.2524
Epoch 4/170
4/4 [==============================] - 0s 72ms/step - loss: 2.0212 - categorical_accuracy: 0.4243
Epoch 5/170
4/4 [==============================] - 0s 67ms/step - loss: 3.4403 - categorical_accuracy: 0.3890
Epoch 6/170
4/4 [==============================] - 0s 70ms/step - loss: 3.0755 - categorical_accuracy: 0.3173
Epoch 7/170
4/4 [==============================] - 0s 69ms/step - loss: 2.0218 - categorical_accuracy: 0.3785
Epoch 8/170
4/4 [==============================] - 0s 67ms/step - loss: 1.2823 - categorical_accuracy: 0.5390
Epoch 9/170
4/4 [==============================] - 0s 68ms/step - loss: 1.1284 - categorical_accuracy: 0.5539
Epoch 10/

4/4 [==============================] - 0s 82ms/step - loss: 0.6206 - categorical_accuracy: 0.7768
Epoch 75/170
4/4 [==============================] - 0s 82ms/step - loss: 0.5771 - categorical_accuracy: 0.7144
Epoch 76/170
4/4 [==============================] - 0s 78ms/step - loss: 0.5578 - categorical_accuracy: 0.7259
Epoch 77/170
4/4 [==============================] - 0s 85ms/step - loss: 0.5149 - categorical_accuracy: 0.7343
Epoch 78/170
4/4 [==============================] - 0s 84ms/step - loss: 0.5177 - categorical_accuracy: 0.7978
Epoch 79/170
4/4 [==============================] - 0s 84ms/step - loss: 0.5367 - categorical_accuracy: 0.7661
Epoch 80/170
4/4 [==============================] - 0s 84ms/step - loss: 0.4931 - categorical_accuracy: 0.8076
Epoch 81/170
4/4 [==============================] - 0s 83ms/step - loss: 0.5968 - categorical_accuracy: 0.7880
Epoch 82/170
4/4 [==============================] - 0s 84ms/step - loss: 0.4296 - categorical_accuracy: 0.8135
Epoch 83/170
4

4/4 [==============================] - 0s 80ms/step - loss: 0.4639 - categorical_accuracy: 0.8411
Epoch 148/170
4/4 [==============================] - 0s 81ms/step - loss: 0.3751 - categorical_accuracy: 0.8671
Epoch 149/170
4/4 [==============================] - 0s 88ms/step - loss: 0.3437 - categorical_accuracy: 0.8588
Epoch 150/170
4/4 [==============================] - 0s 82ms/step - loss: 0.3682 - categorical_accuracy: 0.8601
Epoch 151/170
4/4 [==============================] - 0s 88ms/step - loss: 20.1037 - categorical_accuracy: 0.6835
Epoch 152/170
4/4 [==============================] - 0s 81ms/step - loss: 13.3321 - categorical_accuracy: 0.3630
Epoch 153/170
4/4 [==============================] - 0s 80ms/step - loss: 1.6367 - categorical_accuracy: 0.4010
Epoch 154/170
4/4 [==============================] - 0s 84ms/step - loss: 1.2581 - categorical_accuracy: 0.4738
Epoch 155/170
4/4 [==============================] - 0s 82ms/step - loss: 1.0984 - categorical_accuracy: 0.3933
Epoc

## real time

In [92]:
colors = [(245,117,16), (117,245,16), (16,117,245), (240,128,128)]

def prob_viz(res, actions, input_frame, colors):


    output_frame = input_frame.copy()

    for num, prob in enumerate(res):

        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)

        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        
    return output_frame

In [93]:
sequence = []

sentence = []

predictions = []

threshold = 0.5


cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)

        sequence = sequence[-30:]
        
        if len(sequence) == 30:

            res = model.predict(np.expand_dims(sequence, axis = 0))[0]

            print(actions[np.argmax(res)])

            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 

                if res[np.argmax(res)] > threshold: 

                    if len(sentence) > 0: 

                        if actions[np.argmax(res)] != sentence[-1]:

                            sentence.append(actions[np.argmax(res)])
                    else:

                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 

                sentence = sentence[-5:]
                
            image = prob_viz(res, actions, image, colors)
            

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)

            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            cv2.imshow('openCV Feed', image)
        
            if cv2.waitKey(10) & 0xFF == ord('q'):
        
               break
        
    cap.release()
    cv2.destroyAllWindows()

eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
eating food
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
feeling sleepy
feeling sleepy
sitting actively
sitting actively
sitting actively
sitting activ

eating food
eating food
eating food
eating food
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
smoking cigarette
eating food
smoking cigarette
smoking cigarette
feeling sleepy
feeling sleepy
feeling sl

sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
eating food
eating food
eating food
eating food
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
eating food
eating food
eating food
sitting actively
sitting activel

sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting active

sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting actively
sitting active